## Creación Archivos

In [1]:
import pandas as pd

df = pd.read_csv("../DW/sales.csv", sep=";", encoding="latin1")

products_df = df[['product_id','product_name','category']].drop_duplicates()
customers_df = df[['customer_id','customer_name','customer_city']].drop_duplicates()
reps_df = df[['rep_id','rep_name','region']].drop_duplicates()

sales_df = df[['sale_id','sale_date','customer_id','rep_id']].drop_duplicates()
sales_df['sale_date'] = pd.to_datetime(sales_df['sale_date'], dayfirst=True)

sale_items_df = df[['sale_id','product_id','quantity','unit_price']]

products_df.to_csv("../DW/products.csv", index=False, encoding="utf-8")
customers_df.to_csv("../DW/customers.csv", index=False, encoding="utf-8")
reps_df.to_csv("../DW/reps.csv", index=False, encoding="utf-8")
sales_df.to_csv("../DW/sales.csv", index=False, encoding="utf-8", date_format="%Y-%m-%d")
sale_items_df.to_csv("../DW/sale_items.csv", index=False, encoding="utf-8")

print("Archivos CSV creados correctamente")

Archivos CSV creados correctamente


## DWH - Datamart de ventas

- Se crea una base de datos temporal utilizando duckdb

In [2]:
import duckdb
import pandas as pd
con = duckdb.connect("dwh.duckdb")

### Validación de tablas o vistas creadas

In [13]:
con.execute("""
SELECT table_schema, table_name, table_type
FROM information_schema.tables
ORDER BY table_schema, table_name
""").fetch_df()

,table_schema,table_name,table_type
0,dw,dim_customer,BASE TABLE
1,dw,dim_date,BASE TABLE
2,dw,dim_product,BASE TABLE
3,dw,dim_rep,BASE TABLE
4,dw,fact_sales,BASE TABLE
5,staging,customers,BASE TABLE
6,staging,products,BASE TABLE
7,staging,reps,BASE TABLE
8,staging,sale_items,BASE TABLE
9,staging,sales,BASE TABLE


### Creación de esquemas

- Stagging
- DW

In [4]:
con.execute("""
CREATE SCHEMA staging;
CREATE SCHEMA dw;
""")
print("Esquemas creados exitosamente!")

Esquemas creados exitosamente!


### Tablas stagging

In [5]:
con.execute("""
CREATE TABLE staging.products AS SELECT * FROM read_csv_auto('products.csv', header=True);
CREATE TABLE staging.customers AS SELECT * FROM read_csv_auto('customers.csv', header=True);
CREATE TABLE staging.reps AS SELECT * FROM read_csv_auto('reps.csv', header=True);
CREATE TABLE staging.sales AS SELECT * FROM read_csv_auto('sales.csv', header=True);
CREATE TABLE staging.sale_items AS SELECT * FROM read_csv_auto('sale_items.csv', header=True);
""")
print("Tablas en zona de staging creadas exitosamente!")

Tablas en zona de staging creadas exitosamente!


In [6]:
con.execute("""
SELECT DISTINCT
  TRIM(rep_id)   AS rep_id,
  TRIM(rep_name) AS rep_name,
  TRIM(region)   AS region
FROM staging.reps
WHERE rep_id IS NOT NULL;
""").fetch_df()

,rep_id,rep_name,region
0,R11,Laura Sánchez,Pacífico
1,R10,Juan Pérez,Andina
2,R12,Camila Torres,Pacífico


### Vistas de transformación

- Se limpian y preparan los datos que serán insertados en las dimensiones y tablas de hechos

In [7]:
con.execute("""
CREATE VIEW staging.vw_products AS
SELECT DISTINCT
  TRIM(product_id)   AS product_id,
  TRIM(product_name) AS product_name,
  TRIM(category)     AS category
FROM staging.products
WHERE product_id IS NOT NULL;

CREATE VIEW staging.vw_customers AS
SELECT DISTINCT
  TRIM(customer_id)   AS customer_id,
  TRIM(customer_name) AS customer_name,
  TRIM(customer_city) AS customer_city
FROM staging.customers
WHERE customer_id IS NOT NULL;

CREATE VIEW staging.vw_reps AS
SELECT DISTINCT
  TRIM(rep_id)   AS rep_id,
  TRIM(rep_name) AS rep_name,
  TRIM(region)   AS region
FROM staging.reps
WHERE rep_id IS NOT NULL;

CREATE VIEW staging.vw_sales AS
SELECT
  sale_id,
  CAST(sale_date AS DATE) AS sale_date,
  TRIM(customer_id) AS customer_id,
  TRIM(rep_id) AS rep_id
FROM staging.sales;

CREATE VIEW staging.vw_sale_items AS
SELECT
  sale_id,
  TRIM(product_id) AS product_id,
  CAST(quantity AS INTEGER) AS quantity,
  CAST(unit_price AS DECIMAL(18,2)) AS unit_price
FROM staging.sale_items;
""")
print("Vistas de transformación y limpieza creadas exitosamente!")

Vistas de transformación y limpieza creadas exitosamente!


### Creación de dimensiones

- Product
- Customer
- Rep

In [8]:
con.execute("""
-- Dimensión producto
CREATE TABLE dw.dim_product (
  product_key  INTEGER,
  product_id   VARCHAR,
  product_name VARCHAR,
  category     VARCHAR
);

INSERT INTO dw.dim_product (product_key, product_id, product_name, category)
SELECT
  ROW_NUMBER() OVER (ORDER BY product_id) AS product_key,
  product_id,
  product_name,
  category
FROM staging.vw_products;

-- Luego puedes hacer:
ALTER TABLE dw.dim_product
ADD PRIMARY KEY (product_key);

-- Dimensión cliente
CREATE TABLE dw.dim_customer (
  customer_key  INTEGER,
  customer_id   VARCHAR,
  customer_name VARCHAR,
  customer_city VARCHAR
);

INSERT INTO dw.dim_customer (customer_key, customer_id, customer_name, customer_city)
SELECT
  ROW_NUMBER() OVER (ORDER BY customer_id) AS customer_key,
  customer_id,
  customer_name,
  customer_city
FROM staging.vw_customers;

ALTER TABLE dw.dim_customer
ADD PRIMARY KEY (customer_key);

-- Dimensión rep
CREATE TABLE dw.dim_rep (
  rep_key  INTEGER,
  rep_id   VARCHAR,
  rep_name VARCHAR,
  region   VARCHAR
);

INSERT INTO dw.dim_rep (rep_key, rep_id, rep_name, region)
SELECT
  ROW_NUMBER() OVER (ORDER BY rep_id) AS rep_key,
  rep_id,
  rep_name,
  region
FROM staging.vw_reps;

ALTER TABLE dw.dim_rep
ADD PRIMARY KEY (rep_key);
""")
print("Dimensiones creadas exitosamente!")

Dimensiones creadas exitosamente!


### Dimensión tiempo

In [9]:
df_sales = pd.read_csv('sales.csv', parse_dates=["sale_date"])

min_date = df_sales["sale_date"].min()
max_date = df_sales["sale_date"].max()

dr = pd.date_range(min_date, max_date, freq="D")
df_dim_date = pd.DataFrame({"full_date": dr})
df_dim_date["year"] = df_dim_date["full_date"].dt.year
df_dim_date["month"] = df_dim_date["full_date"].dt.month
df_dim_date["day"] = df_dim_date["full_date"].dt.day
df_dim_date["day_name"] = df_dim_date["full_date"].dt.day_name()
df_dim_date["week"] = df_dim_date["full_date"].dt.isocalendar().week
df_dim_date["quarter"] = df_dim_date["full_date"].dt.quarter
df_dim_date["date_key"] = df_dim_date["full_date"].dt.strftime("%Y%m%d").astype(int)
df_dim_date = df_dim_date[[
    "date_key", "full_date", "year", "month", "day",
    "day_name", "week", "quarter"
]]

con.execute("""
CREATE TABLE dw.dim_date AS
SELECT
  date_key,
  full_date,
  year,
  month,
  day,
  day_name,
  week,
  quarter
FROM df_dim_date
""")
print("Dimensión de tiempo y fechas creada exitosamente!")

Dimensión de tiempo y fechas creada exitosamente!


### Tabla de hechos

In [10]:
con.execute("""
CREATE TABLE dw.fact_sales (
  sale_id      INTEGER,
  date_key     INTEGER,
  customer_key INTEGER,
  product_key  INTEGER,
  rep_key      INTEGER,
  quantity     INTEGER,
  unit_price   DECIMAL(18,2),
  total_sales  DECIMAL(18,2),
  PRIMARY KEY (sale_id, product_key)
);

INSERT INTO dw.fact_sales
SELECT
  si.sale_id,
  dd.date_key,
  dc.customer_key,
  dp.product_key,
  dr.rep_key,
  si.quantity,
  si.unit_price,
  si.quantity * si.unit_price
FROM staging.vw_sale_items si
JOIN staging.vw_sales s        ON si.sale_id = s.sale_id
JOIN dw.dim_date dd    ON dd.full_date = s.sale_date
JOIN dw.dim_customer dc ON dc.customer_id = s.customer_id
JOIN dw.dim_product dp  ON dp.product_id  = si.product_id
JOIN dw.dim_rep dr      ON dr.rep_id      = s.rep_id;
""")
print("Tabla de hechos creada exitosamente!!")

Tabla de hechos creada exitosamente!!


In [11]:
con.execute("SELECT * FROM dw.fact_sales").fetchdf()

,sale_id,date_key,customer_key,product_key,rep_key,quantity,unit_price,total_sales
0,1001,20250701,1,1,1,2,1200.0,2400.0
1,1002,20250702,2,2,2,1,455.0,455.0
2,1003,20250702,1,3,1,1,3500.0,3500.0
3,1004,20250703,3,4,3,3,900.0,2700.0
4,1005,20250704,4,1,2,1,1200.0,1200.0
5,1006,20250705,5,4,1,2,900.0,1800.0
6,1007,20250706,6,2,3,4,455.0,1820.0
7,1008,20250707,7,3,2,2,3500.0,7000.0
8,1009,20250708,8,5,1,1,700.0,700.0
9,1010,20250709,9,6,2,2,1100.0,2200.0


¿Cuál es la evolución mensual de ingresos y unidades vendidas por categoría de producto?

¿Qué clientes generan más ingresos totales?

¿Cómo se desempeñan los representantes de ventas (y por región) en términos de ingresos, número de ventas y unidades vendidas?

¿Cómo ha cambiado año con año el ingreso de una categoría específica de producto?

En un mes dado(lo puede elegir usted), ¿cuál es el promedio diario de ventas y de ingresos por cliente?

In [12]:
con.execute("").fetchdf()

AttributeError: 'NoneType' object has no attribute 'fetchdf'